## Downloading stock market data from the CVM website.
## The data will consist of quarterly reports spanning from 2012 to 2022.

### Setting the display

In [26]:
import pandas as pd

#pd.set_option('display.max_columns', 10)
pd.reset_option('display.max_columns')
pd.set_option('display.width', 200)
#pd.reset_option('display.width')

### Installing libs

In [27]:
%pip install requests beautifulsoup4
%pip install chardet
%pip install files
%pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Downloading balance sheets files

In [28]:
import os
import requests
from urllib.parse import urljoin
import zipfile
import chardet

# Function to check if a file's encoding is UTF-8
def is_utf8(file_path):
    try:
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            result = chardet.detect(raw_data)
            return result['encoding'] == 'utf-8'
    except Exception:
        return False

# URL of the website containing the ZIP files
base_url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

# Create a directory to save the downloaded files
download_dir = "downloaded_files"
os.makedirs(download_dir, exist_ok=True)

# Define the years you want to download (2012 to 2022)
years_to_download = set(str(year) for year in range(2012, 2023))

# Send an HTTP GET request to the URL
response = requests.get(base_url)
response.raise_for_status()

# Parse the HTML content using BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all links on the page
links = soup.find_all("a")

# Iterate through the links and download ZIP files for the specified years
for link in links:
    file_url = urljoin(base_url, link["href"])
    
    # Example: Assuming ZIP files are named like "itr_cia_aberta_YEAR.zip"
    if file_url.endswith(".zip"):
        zip_file_name = os.path.basename(file_url)
        
        # Extract the year from the ZIP file name
        year_part = zip_file_name.split("_")[-1].split(".")[0]
        
        # Check if the ZIP file is from a year within the specified range
        if year_part in years_to_download:
            # Check if the ZIP file already exists
            if not os.path.exists(os.path.join(download_dir, zip_file_name)):
                print(f"Downloading: {zip_file_name}")
                with open(os.path.join(download_dir, zip_file_name), "wb") as file:
                    file_response = requests.get(file_url)
                    file.write(file_response.content)
            else:
                print(f"File already exists: {zip_file_name}")

# Process the downloaded ZIP files (extract, remove "ind" files, and convert)
for zip_file_name in os.listdir(download_dir):
    if zip_file_name.endswith(".zip"):
        zip_file_path = os.path.join(download_dir, zip_file_name)
        zip_subdir = os.path.splitext(zip_file_name)[0]  # Use ZIP file name without extension as subdirectory name
        zip_subdir_path = os.path.join(download_dir, zip_subdir)

        # Check if the ZIP file has already been extracted
        if not os.path.exists(zip_subdir_path):
            print(f"Unzipping: {zip_file_name} -> {zip_subdir_path}")
            with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(zip_subdir_path)

            # Remove files with "ind" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "ind" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Remove files with "itr_cia_aberta_20" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "itr_cia_aberta_20" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Remove files with "itr_cia_aberta_DFC_MD_con_20" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "itr_cia_aberta_DFC_MD_con_20" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Convert CSV files from ISO-8859-1 to UTF-8 only if they are not already UTF-8
            for csv_file_name in os.listdir(zip_subdir_path):
                if csv_file_name.endswith(".csv"):
                    csv_file_path = os.path.join(zip_subdir_path, csv_file_name)
                    if not is_utf8(csv_file_path):
                        print(f"Converting encoding: {csv_file_path}")
                        with open(csv_file_path, 'r', encoding='ISO-8859-1') as source_file:
                            content = source_file.read()
                        with open(csv_file_path, 'w', encoding='utf-8') as target_file:
                            target_file.write(content)
                    else:
                        print(f"File already in UTF-8: {csv_file_path}")

print("Download, extraction, file removal, and encoding conversion completed.")


File already exists: itr_cia_aberta_2012.zip
File already exists: itr_cia_aberta_2013.zip
File already exists: itr_cia_aberta_2014.zip
File already exists: itr_cia_aberta_2015.zip
File already exists: itr_cia_aberta_2016.zip
File already exists: itr_cia_aberta_2017.zip
File already exists: itr_cia_aberta_2018.zip
File already exists: itr_cia_aberta_2019.zip
File already exists: itr_cia_aberta_2020.zip
File already exists: itr_cia_aberta_2021.zip
File already exists: itr_cia_aberta_2022.zip
Download, extraction, file removal, and encoding conversion completed.


### Creating the datasets

In [29]:
import numpy as np
import re

bpa_df = None
bpp_df = None
dfc_mi_df = None
dmpl_df = None
dra_df = None
dre_df = None
dva_df = None

d_parser = lambda x: pd.datetime.strptime(format='%Y-%m-%d')

# Load files into Data Frames
for dir_name in os.listdir(download_dir):
    if not dir_name.endswith(".zip"):
        dir_path = os.path.join(download_dir, dir_name)

        for csv_file_name in os.listdir(dir_path):
            csv_file_path = os.path.join(dir_path, csv_file_name)
            print("\n-------------------------------------------------------")
            print(f"Reading csv file: {csv_file_path}")

            csv_file_type = os.path.splitext(csv_file_name)[0][15:19].replace("_", "")
            print(f"CSV type: {csv_file_type}")

            local_df = pd.read_csv(csv_file_path, sep=';')

            local_df['DT_REFER'] = pd.to_datetime(local_df['DT_REFER'], format='%Y-%m-%d')
            local_df['DT_FIM_EXERC'] = pd.to_datetime(local_df['DT_FIM_EXERC'], format='%Y-%m-%d')

            if ((csv_file_type == 'BPA') or (csv_file_type == 'BPP')):
                # Subtract 3 months from the 'DT_FIM_EXERC' using vectorized operations
                local_df['DT_INI_EXERC'] = local_df['DT_FIM_EXERC'] - pd.DateOffset(months=3)

                # Set the day to the 1st day of the month using pd.offsets.MonthBegin(1)
                local_df['DT_INI_EXERC'] = local_df['DT_INI_EXERC'] + pd.offsets.MonthBegin(1)

                # Reorder the columns to move 'DT_INI_EXERC' to the 8th position
                cols = list(local_df.columns)
                cols.remove('DT_INI_EXERC')
                cols.insert(8, 'DT_INI_EXERC')
                local_df = local_df[cols]
            else:
                local_df['DT_INI_EXERC'] = pd.to_datetime(local_df['DT_INI_EXERC'], format='%Y-%m-%d')

            print(f"Dataframe size: {len(local_df)}")

            match csv_file_type:
                case 'BPA':
                    if bpa_df is None:
                        bpa_df = local_df
                    else:
                        bpa_df = pd.concat([bpa_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpa_df)}")

                case 'BPP':
                    if bpp_df is None:
                        bpp_df = local_df
                    else:
                        bpp_df = pd.concat([bpp_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpp_df)}")

                case 'DFC':
                    if dfc_mi_df is None:
                        dfc_mi_df = local_df
                    else:
                        dfc_mi_df = pd.concat([dfc_mi_df, local_df])
                    print(f"Dataframe size - after concat: {len(dfc_mi_df)}")

                case 'DMPL':
                    if dmpl_df is None:
                        dmpl_df = local_df
                    else:
                        dmpl_df = pd.concat([dmpl_df, local_df])
                    print(f"Dataframe size - after concat: {len(dmpl_df)}")

                case 'DRA':
                    if dra_df is None:
                        dra_df = local_df
                    else:
                        dra_df = pd.concat([dra_df, local_df])
                    print(f"Dataframe size - after concat: {len(dra_df)}")

                case 'DRE':
                    if dre_df is None:
                        dre_df = local_df
                    else:
                        dre_df = pd.concat([dre_df, local_df])
                    print(f"Dataframe size - after concat: {len(dre_df)}")

                case 'DVA':
                    if dva_df is None:
                        dva_df = local_df
                    else:
                        dva_df = pd.concat([dva_df, local_df])
                    print(f"Dataframe size - after concat: {len(dva_df)}")



-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_BPA_con_2012.csv
CSV type: BPA
Dataframe size: 132416
Dataframe size - after concat: 132416

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_BPP_con_2012.csv
CSV type: BPP
Dataframe size: 224976
Dataframe size - after concat: 224976

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DFC_MI_con_2012.csv
CSV type: DFC
Dataframe size: 80864
Dataframe size - after concat: 80864

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DMPL_con_2012.csv
CSV type: DMPL
Dataframe size: 458552
Dataframe size - after concat: 458552

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_a

### Data analising

In [30]:
def print_df(df_name, df):
    print(f'\n\n{df_name}: {len(df):,} records')
    print('-------------------------------------------------------------------')
    print('\t\t\t TYPES')
    print(df.dtypes)

    print('\t\t\t NULL VALUES')
    print('CNPJ_CIA: ' + str(df['CNPJ_CIA'].isnull().sum().sum()))
    print('DT_REFER: ' + str(df['DT_REFER'].isnull().sum().sum()))
    print('VERSAO: ' + str(df['VERSAO'].isnull().sum().sum()))
    print('DENOM_CIA: ' + str(df['DENOM_CIA'].isnull().sum().sum()))
    print('CD_CVM: ' + str(df['CD_CVM'].isnull().sum().sum()))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].isnull().sum().sum()))
    print('MOEDA: ' + str(df['MOEDA'].isnull().sum().sum()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].isnull().sum().sum()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].isnull().sum().sum()))
    print('DT_INI_EXERC: ' + str(df['DT_INI_EXERC'].isnull().sum().sum()))
    print('DT_FIM_EXERC: ' + str(df['DT_FIM_EXERC'].isnull().sum().sum()))
    print('CD_CONTA: ' + str(df['CD_CONTA'].isnull().sum().sum()))
    print('DS_CONTA: ' + str(df['DS_CONTA'].isnull().sum().sum()))
    print('VL_CONTA: ' + str(df['VL_CONTA'].isnull().sum().sum()))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].isnull().sum().sum()))

    print('\t\t\t COUNT UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(len(df['VERSAO'].unique())))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(len(df['GRUPO_DFP'].unique())))
    print('MOEDA: ' + str(len(df['MOEDA'].unique())))
    print('ESCALA_MOEDA: ' + str(len(df['ESCALA_MOEDA'].unique())))
    print('ORDEM_EXERC: ' + str(len(df['ORDEM_EXERC'].unique())))
    print('DT_INI_EXERC: ' + str(len(df['DT_INI_EXERC'].unique())))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(len(df['ST_CONTA_FIXA'].unique())))

    print('\t\t\t UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(df['VERSAO'].unique()))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].unique()))
    print('MOEDA: ' + str(df['MOEDA'].unique()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].unique()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].unique()))
    print('DT_INI_EXERC: ' + str(len(df['DT_INI_EXERC'].unique())))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].unique()))


print_df('bpa_df', bpa_df)
print_df('bpp_df', bpp_df)
print_df('dfc_mi_df', dfc_mi_df)
print_df('dmpl_df', dmpl_df)
print_df('dra_df', dra_df)
print_df('dre_df', dre_df)
print_df('dva_df', dva_df)




bpa_df: 1,529,932 records
-------------------------------------------------------------------
			 TYPES
CNPJ_CIA                 object
DT_REFER         datetime64[ns]
VERSAO                    int64
DENOM_CIA                object
CD_CVM                    int64
GRUPO_DFP                object
MOEDA                    object
ESCALA_MOEDA             object
DT_INI_EXERC     datetime64[ns]
ORDEM_EXERC              object
DT_FIM_EXERC     datetime64[ns]
CD_CONTA                 object
DS_CONTA                 object
VL_CONTA                float64
ST_CONTA_FIXA            object
dtype: object
			 NULL VALUES
CNPJ_CIA: 0
DT_REFER: 0
VERSAO: 0
DENOM_CIA: 0
CD_CVM: 0
GRUPO_DFP: 0
MOEDA: 0
ESCALA_MOEDA: 0
ORDEM_EXERC: 0
DT_INI_EXERC: 0
DT_FIM_EXERC: 0
CD_CONTA: 0
DS_CONTA: 0
VL_CONTA: 0
ST_CONTA_FIXA: 0
			 COUNT UNIQUE VALUES
CNPJ_CIA: 634
DT_REFER: 63
VERSAO: 6
DENOM_CIA: 676
CD_CVM: 636
GRUPO_DFP: 1
MOEDA: 1
ESCALA_MOEDA: 2
ORDEM_EXERC: 2
DT_INI_EXERC: 72
DT_FIM_EXERC: 78
CD_CONTA: 406


### Data cleansing

In [33]:
bpa_df_backup = bpa_df
bpp_df_backup = bpp_df
dfc_mi_df_backup = dfc_mi_df
dmpl_df_backup = dmpl_df
dra_df_backup = dra_df
dre_df_backup = dre_df
dva_df_backup = dva_df

# bpa_df = bpa_df_backup
# bpp_df = bpp_df_backup
# dfc_mi_df = dfc_mi_df_backup
# dmpl_df = dmpl_df_backup
# dra_df = dra_df_backup
# dre_df = dre_df_backup
# dva_df = dva_df_backup

In [32]:
# calculate value according to field ESCALA_MOELA
def set_vl_conta(df_name, df):
    print(f'setting vl_conta based on escala_moeda: {df_name}')
    df['VL_CONTA'] = np.where(df['ESCALA_MOEDA'] == 'MIL', df['VL_CONTA'] * 1000, df['VL_CONTA'])
    return df

def remove_non_mandatory_columns(df_name, df):
    print(f'removing non mandatory columns from: {df_name}')
    existing_cols = df.columns.intersection(['CNPJ_CIA', 'VERSAO', 'DENOM_CIA', 'MOEDA', 'ESCALA_MOEDA', 'COLUNA_DF', 'ST_CONTA_FIXA'])
    return df.drop(columns=existing_cols, axis=1)

def remove_non_used_accounts(df_name, df):
    print(f'removing non used accounts for: {df_name}')

    account_to_be_kept = ['1','1.01','1.01.01','1.01.03','1.01.04','1.02','1.02.03','1.02.03.01','1.02.03.02', 
                        '2.01','2.01.02','2.01.03','2.01.04', 
                        '3.01','3.02','3.03','3.04','3.04.01','3.04.02','3.04.03','3.04.04','3.04.05','3.04.06','3.06','3.06.01','3.06.02','3.08','3.08.01','3.08.02','3.11','3.11.01', 
                        '5.04.06','5.05.01', 
                        '6.01','6.01.01','6.02','6.03','6.05','6.05.01','6.05.02', 
                        '7.04.01']
    return df[df['CD_CONTA'].isin(account_to_be_kept)]

def select_companies(df_name, df):
    print(f'selecting companies for: {df_name}')
    print('companies: CPFE3(18660), CMIG4(2453), ELET6(2437), ENEV3(21237), TAEE11(20257)')

    companies = [18660, 2453, 2437, 21237, 20257]
    df['CD_CVM'] = df[df['CD_CVM'].isin(companies)]
    return df

print("\n")
bpa_df = set_vl_conta('bpa_df', bpa_df)
bpp_df = set_vl_conta('bpp_df', bpp_df)
dfc_mi_df = set_vl_conta('dfc_mi_df', dfc_mi_df)
dmpl_df = set_vl_conta('dmpl_df', dmpl_df)
dra_df = set_vl_conta('dra_df', dra_df)
dre_df = set_vl_conta('dre_df', dre_df)
dva_df = set_vl_conta('dva_df', dva_df)

print("\n")
bpa_df = remove_non_mandatory_columns('bpa_df', bpa_df)
bpp_df = remove_non_mandatory_columns('bpp_df', bpp_df)
dfc_mi_df = remove_non_mandatory_columns('dfc_mi_df', dfc_mi_df)
dmpl_df = remove_non_mandatory_columns('dmpl_df', dmpl_df)
dra_df = remove_non_mandatory_columns('dra_df', dra_df)
dre_df = remove_non_mandatory_columns('dre_df', dre_df)
dva_df = remove_non_mandatory_columns('dva_df', dva_df)

print("\n")
bpa_df = remove_non_used_accounts('bpa_df', bpa_df)
bpp_df = remove_non_used_accounts('bpp_df', bpp_df)
dfc_mi_df = remove_non_used_accounts('dfc_mi_df', dfc_mi_df)
dmpl_df = remove_non_used_accounts('dmpl_df', dmpl_df)
dra_df = remove_non_used_accounts('dra_df', dra_df)
dre_df = remove_non_used_accounts('dre_df', dre_df)
dva_df = remove_non_used_accounts('dva_df', dva_df)

# print("\n")
# bpa_df = select_companies('bpa_df', bpa_df)
# bpp_df = select_companies('bpp_df', bpp_df)
# dfc_mi_df = select_companies('dfc_mi_df', dfc_mi_df)
# dmpl_df = select_companies('dmpl_df', dmpl_df)
# dra_df = select_companies('dra_df', dra_df)
# dre_df = select_companies('dre_df', dre_df)
# dva_df = select_companies('dva_df', dva_df)




setting vl_conta based on escala_moeda: bpa_df
setting vl_conta based on escala_moeda: bpp_df
setting vl_conta based on escala_moeda: dfc_mi_df
setting vl_conta based on escala_moeda: dmpl_df
setting vl_conta based on escala_moeda: dra_df
setting vl_conta based on escala_moeda: dre_df
setting vl_conta based on escala_moeda: dva_df


removing non mandatory columns from: bpa_df
removing non mandatory columns from: bpp_df
removing non mandatory columns from: dfc_mi_df
removing non mandatory columns from: dmpl_df
removing non mandatory columns from: dra_df
removing non mandatory columns from: dre_df
removing non mandatory columns from: dva_df


removing non used accounts for: bpa_df
removing non used accounts for: bpp_df
removing non used accounts for: dfc_mi_df
removing non used accounts for: dmpl_df
removing non used accounts for: dra_df
removing non used accounts for: dre_df
removing non used accounts for: dva_df


selecting companies for: bpa_df
companies: CPFE3(18660), CMIG4(2453), 

ValueError: Columns must be same length as key

### Data transformation

#### Merge dataframes into one

In [ ]:
df = pd.concat([bpa_df, bpp_df, dfc_mi_df, dmpl_df, dra_df, dre_df, dva_df])


#### Create function and operation to use for ORDEM_EXERC of value 'PENÚLTIMO'

In [ ]:
# Reorder the columns to move 'ORDEM_EXERC' to the last position
cols = list(df.columns)
cols.remove('ORDEM_EXERC')
cols.insert(-1, 'ORDEM_EXERC')
df = df[cols]

# Reorder the columns to move 'CD_CONTA' to the 2nd position
cols = list(df.columns)
cols.remove('CD_CONTA')
cols.insert(2, 'CD_CONTA')
df = df[cols]

def createPenultimo(columnName, position, df, objectType):
    df[columnName + '_PENULTIMO'] = np.where(
        df['ORDEM_EXERC'] == 'PENÚLTIMO', 
        df[columnName], 
        None)

    # remove this same values from column
    df[columnName] = np.where(
        df['ORDEM_EXERC'] != 'PENÚLTIMO', 
        df[columnName], 
        None)
    
    match objectType:
        case 'DATE_TIME':
            df[columnName + '_PENULTIMO'] = pd.to_datetime(df[columnName + '_PENULTIMO'], format='%Y-%m-%d')
            df[columnName] = pd.to_datetime(df[columnName], format='%Y-%m-%d')
        case 'FLOAT':
            df[columnName + '_PENULTIMO'] = df[columnName + '_PENULTIMO'].astype(float)
            df[columnName] = df[columnName].astype(float)
        # case 'STRING':
        #     df[columnName + '_PENULTIMO'] = df[columnName + '_PENULTIMO'].astype(str)
        #     df[columnName] = df[columnName].astype(str)

    # Reorder the columns to move to correct position
    cols = list(df.columns)
    cols.remove(columnName + '_PENULTIMO')
    cols.insert(position, columnName + '_PENULTIMO')
    df = df[cols]

    return df

#### Create column 'DT_INI_EXERC_PENULTIMO'

In [ ]:
df = createPenultimo('DT_INI_EXERC', 4, df, 'DATE_TIME')

print(df.dtypes)
df.head()


#### Create column 'DT_FIM_EXERC_PENULTIMO'

In [ ]:
df = createPenultimo('DT_FIM_EXERC', 6, df, 'DATE_TIME')
print(df.dtypes)

df.head()


#### Create column 'VL_CONTA_PENULTIMO'

In [ ]:
df = createPenultimo('VL_CONTA', 10, df, 'FLOAT')

print(df.dtypes)
df.head()

#### Remove disposable columns

In [ ]:
cols = df.columns.intersection(['GRUPO_DFP', 'DS_CONTA', 'ORDEM_EXERC'])
df = df.drop(columns=cols, axis=1)

print(df.dtypes)
df.head()

#### Group by 'DT_REFER', 'CD_CVM', and 'CD_CONTA' and aggregate the values

In [ ]:
agg_funcs = {
    'DT_INI_EXERC_PENULTIMO': 'first',
    'DT_INI_EXERC': 'first',
    'DT_FIM_EXERC_PENULTIMO': 'first',
    'DT_FIM_EXERC': 'first',
    'VL_CONTA_PENULTIMO': 'last',
    'VL_CONTA': 'last'
}

df = df.groupby(['DT_REFER', 'CD_CVM', 'CD_CONTA']).agg(agg_funcs).reset_index()

# Reorder the columns as needed
df = df[['DT_REFER', 'CD_CVM', 'CD_CONTA', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC', 'VL_CONTA_PENULTIMO', 'VL_CONTA']]

# View the final DataFrame
print(df.dtypes)
df.head()

In [ ]:
df_backup = df
#df = df_backup

#### Transposing data from rows to columns based on CD_CONTA

In [ ]:
# replicating CD_CONTA column to match to VL_CONTA_PENULTIMO column
df['CD_CONTA_PENULTIMO'] = df['CD_CONTA']
# Reorder the columns to move 'CD_CONTA' to the 2nd position
cols = list(df.columns)
cols.remove('CD_CONTA_PENULTIMO')
cols.insert(2, 'CD_CONTA_PENULTIMO')
df = df[cols]

In [ ]:
# group data
print("\n")
print(f'grouping rows')
#df_pivot = df.pivot_table(index=['DT_REFER', 'CD_CVM', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'], columns=['CD_CONTA_PENULTIMO', 'CD_CONTA'], values=['VL_CONTA_PENULTIMO', 'VL_CONTA'], dropna=True).reset_index()

#df_pivot = df.groupby(['DT_REFER', 'CD_CVM', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'])

#df_pivot = df.pivot_table(index=['DT_REFER', 'CD_CVM', 'CD_CONTA', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC']).reset_index()


#------------------------

# Combine 'CD_CONTA' and 'VL_CONTA' to create new column names
# df['CD_CONTA'] = df['CD_CONTA'].astype(str)
# df['CD_CONTA'] = df['CD_CONTA'] + '_' + df['VL_CONTA'].astype(str)

# # Pivot the DataFrame
# pivot_df = df.pivot(index=['DT_REFER', 'CD_CVM', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'],
#                     columns='CD_CONTA',
#                     values=['VL_CONTA_PENULTIMO', 'VL_CONTA']).reset_index()

# # Flatten the multi-level columns
# pivot_df.columns = [' '.join(col).strip() for col in pivot_df.columns.values]

# # Drop the index name
# pivot_df.index.name = None

# # Fill NaN values with empty strings
# pivot_df = pivot_df.fillna('')

#------------------------


# DT_REFER	CD_CVM	CD_CONTA	DT_INI_EXERC_PENULTIMO	DT_INI_EXERC	DT_FIM_EXERC_PENULTIMO	DT_FIM_EXERC	VL_CONTA_PENULTIMO	VL_CONTA
# 0	2012-03-31	94	1	2011-10-01	2012-01-01	2011-12-31	2012-03-31	294757000.0	314589000.0
# 1	2012-03-31	94	1.01	2011-10-01	2012-01-01	2011-12-31	2012-03-31	225588000.0	241973000.0
# 2	2012-03-31	94	1.01.01	2011-10-01	2012-01-01	2011-12-31	2012-03-31	2649000.0	1825000.0
# 3	2012-03-31	94	1.01.03	2011-10-01	2012-01-01	2011-12-31	2012-03-31	57240000.0	68195000.0
# 4	2012-03-31	94	1.01.04	2011-10-01	2012-01-01	2011-12-31	2012-03-31	57123000.0	58077000.0

df['CD_CONTA'] = df['CD_CONTA'].astype(str)
# Combine 'CD_CONTA' and 'VL_CONTA_PENULTIMO' to create new column name
df['CD_CONTA__VL_CONTA_PENULTIMO'] = df['CD_CONTA'] + '__' + df['VL_CONTA_PENULTIMO'].astype(str)
# Combine 'CD_CONTA' and 'VL_CONTA' to create new column name
df['CD_CONTA__VL_CONTA'] = df['CD_CONTA'] + '__' + df['VL_CONTA'].astype(str)

# Pivot the DataFrame
pivot_df = df.pivot_table(index=['DT_REFER', 'CD_CVM', 'DT_INI_EXERC_PENULTIMO', 'DT_INI_EXERC', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'], 
                          columns=['CD_CONTA__VL_CONTA_PENULTIMO', 'CD_CONTA__VL_CONTA'], 
                          values=['VL_CONTA_PENULTIMO', 'VL_CONTA'])

# Flatten the multi-level columns
pivot_df.columns = [' '.join(col).strip() for col in pivot_df.columns.values]

# Reset the index
pivot_df = pivot_df.reset_index()

#------------------------

print('\n')

print('\ndtypes:')
print(pivot_df.dtypes)
print('\nshape:')
print(pivot_df.shape)
print('\ncolumns:')
print(pivot_df.columns)

print('\n')
pivot_df.head(30)